In [35]:
#Below is the code for WMA 
import numpy as np
import random
from pprint import pprint
import pandas as pd
#Code for setting up the data so that we can start to build the model
import random
import time
import csv 
import json
import math
from IPython.display import display
import io
file='temperature_data.csv'

data=pd.DataFrame()
data=pd.read_csv('temperature_data.csv',header=0)
Days=data['days']
weather_com_pred=data['weather.com prediction']
weather_com_pred=list(weather_com_pred)
# print(weather_com_pred)
accu_w_com=data['accuweather.com prediction']
accu_w_com=list(accu_w_com)
# print(accu_w_com)
w_ground_com=data['wunderground.com prediction']
w_ground_com=list(w_ground_com)
# print(w_ground_com)
actual_temp=data['actual temperature']
actual_temp=list(actual_temp)
expert_list=[weather_com_pred,accu_w_com,w_ground_com]
# print(actual_temp)
# print(data.head())
# print(type(accu_w_com))
# print(expert_list)


In [32]:

# parameters
N = len(expert_list)# of experts
eps = 0.13 #epsilon
T = len(Days) #total number of time periods
# notations
# w: np array of expert weights
# f: np array of predictions made by experts
# t: time period from 1...T
# x: decision
# y: outcome of decision x
# M: np array of # mistakes made by experts
# M_wma: total mistakes made by WMA
p=[]
for t in expert_list:
    
    for x in t:
        if x<=70:
            x=1
            p.append(x)
        else:
            x=0
            p.append(x)
a_temp=[]
for a in actual_temp:
    if a<=70:
        a=1
        a_temp.append(a)
    else:
        a=0
        a_temp.append(a)
            
weather_com_pred=p[:61] 
accu_w_com=p[61:122]
w_ground_com=p[122::]
# print(x)
# print(len(x))
# print(y)
# print(len(y))
# print(z)
# print(len(z))





In [33]:
# initialize weight
w = np.ones(N)
M = np.zeros(N)
M_wma = 0
mistakes_list = []
regret = 0
# iterations
for t in range (T):
    print('round: ', t)
    print('expert weights: ', w)
    print('mistakes: ', M)
    mistakes_list.append(list(w))
    
    # experts make predictions, suppose these are random predictions
    f = [weather_com_pred[t],accu_w_com[t],w_ground_com[t]]
    # tally weighted sum of {0,1} predictions  
    one = np.dot(w,f)
    zero = w.sum() - one
    if one>=zero:
        x=1
    else:
        x=0
    
#     x = 1 if any(one) >= zero else 0 # make majority based decision 
    
    # reveal outcome, suppose it is random
    y = a_temp[t]
    M_wma += (x != y) #count number of mistakes
    
    
    # weight update
    for i in range(N):
        w[i] = w[i]*(1 - eps)**(f[i]!=y) # penalize weight if made wrong prediction
        M[i] =M[i]+(f[i] != y)

print('Regret is:', M_wma - M.min())
print('best expert made ', M.min(), ' mistakes')
print('WMA made ', M_wma, ' mistakes')

pd.DataFrame(mistakes_list, columns = ['weather','accuweather','wunderground']).to_csv('WMA_table.csv')

round:  0
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  1
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  2
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  3
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  4
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  5
expert weights:  [1.   1.   0.87]
mistakes:  [0. 0. 1.]
round:  6
expert weights:  [1.   1.   0.87]
mistakes:  [0. 0. 1.]
round:  7
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  8
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  9
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  10
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  11
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  12
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  13
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  14
expert weights:  [0.87   0.7569 0.7569]
mistakes:  [1. 2. 2.]
round:  15
expert weights:  [

In [34]:
w = np.ones(N)
M = np.zeros(N)
M_rwma = 0
prob_list=[]
mistakes_list = []
regret = 0

# iterations
for t in range (T):
    print('round: ', t)
    print('expert weights: ', w)
    print('mistakes: ', M)
    mistakes_list.append(list(w))
    
    # experts make predictions, suppose these are random predictions
    f = [weather_com_pred[t],accu_w_com[t],w_ground_com[t]]
    
    
    
    
    # tally weighted sum of {0,1} predictions  
    one = np.dot(w,f)
    zero = w.sum() - one
    if one>=zero:
        x=1
    else:
        x=0
    
#     x = 1 if any(one) >= zero else 0 # make majority based decision 
    probability = w / w.sum()
    x = np.random.choice(f,1,p = probability)
    # reveal outcome, suppose it is random
    y = a_temp[t]
    M_rwma += (x != y) #count number of mistakes
    
#     probability=one/w.sum()
#     prob_list.append(probability)
    
    
    # weight update
    for i in range(N):
        w[i] = w[i]*(1 - eps)**(f[i]!=y) # penalize weight if made wrong prediction
        M[i] =M[i]+(f[i] != y)


print('Regret is:', M_rwma - M.min())
print('best expert made ', M.min(), ' mistakes')
print('RWMA made ', M_rwma, ' mistakes')
pd.DataFrame(mistakes_list, columns = ['weather','accuweather','wunderground']).to_csv('RWMA_table.csv')

round:  0
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  1
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  2
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  3
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  4
expert weights:  [1. 1. 1.]
mistakes:  [0. 0. 0.]
round:  5
expert weights:  [1.   1.   0.87]
mistakes:  [0. 0. 1.]
round:  6
expert weights:  [1.   1.   0.87]
mistakes:  [0. 0. 1.]
round:  7
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  8
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  9
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  10
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  11
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  12
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  13
expert weights:  [1.   0.87 0.87]
mistakes:  [0. 1. 1.]
round:  14
expert weights:  [0.87   0.7569 0.7569]
mistakes:  [1. 2. 2.]
round:  15
expert weights:  [